In [1]:
from collections import defaultdict
import uuid
import time
import heapq
import datetime
import json

In [2]:
class Order:
    def __init__(self,instrument,order_id,side,price,quantity,timestamp):
        self.instrument = instrument
        self.order_id = order_id
        self.side = side
        self.price = price
        self.quantity = quantity
        self.timestamp = timestamp
        self.filled_quantity = 0
        
    # price/time priority
    def __lt__(self,other):
        if self.side == 'buy':
            return (self.price, -self.timestamp) > (other.price,-other.timestamp) # to get most recent buy order
        else:
            return (self.price,self.timestamp) < (other.price, other.timestamp)
           

In [3]:
class OrderBook:
    def __init__(self):
        self.bids = []
        self.asks = []s
        
    def add(self,order):
        if order.side =='buy':
            heapq.heappush(self.bids,(-order.price,order.timestamp,order.order_id,order.quantity,order))
        elif order.side == 'sell':
            heapq.heappush(self.asks,(order.price,order.timestamp,order.order_id,order.quantity,order))
            
            
    def cancel(self, order):
        if order.side == 'buy':
            self.bids.remove((-order.price,order.timestamp,order.order_id,order.quantity,order))
            heapq.heapify(self.bids)
       
        else:
            self.asks.remove((order.price,order.timestamp,order.order_id,order.quantity,order)) 
            heapq.heapify(self.asks)
            
    def removeafterTrade(self,order):
        
        if order.side == 'buy':
            
            if order.quantity == 0:
                self.bids.remove((-order.price,order.timestamp,order.order_id,order.quantity,order))
                heapq.heapify(self.bids)
       
        else:
            if order.quantity == 0:
                self.asks.remove((order.price,order.timestamp,order.order_id,order.quantity,order)) 
                heapq.heapify(self.asks)
            
            
    def bestBid(self):
        return self.bids[0][4] if self.bids else None

    def bestAsk(self):
        return self.asks[0][4] if self.asks else None
    
    def showBestBidAndAsk(self):
        best_bid = self.bestBid()
        best_ask = self.bestAsk()

        if (best_bid is not None and best_bid.quantity>0):
            print(f"Best Bid: Instrument: {best_bid.instrument}, Price: {best_bid.price}, Quantity: {best_bid.quantity}")
            print("----------------------------------------------------------------------------------------------------")
        else:
            print("No best bid available.")
            print("----------------------------------------------------------------------------------------------------")

        if (best_ask is not None and best_ask.quantity>0):
            print(f"Best Ask: Instrument: {best_ask.instrument}, Price: {best_ask.price}, Quantity: {best_ask.quantity}")
            print("----------------------------------------------------------------------------------------------------")
        else:
            print("No best ask available.")
            print("----------------------------------------------------------------------------------------------------")

In [4]:
class Trade:
    def __init__(self,id,name,price,volume):
        self.id = id
        self.name = name
        self.price = price
        self.volume = volume

In [5]:
class MatchedOrder:
    def __init__(self, order_id, side, price, filled_quantity, instrument, timestamp):
        self.order_id = order_id
        self.side = side
        self.price = price
        self.filled_quantity = filled_quantity
        self.instrument = instrument
        self.timestamp = timestamp

In [6]:
# class MatchedOrder:
#     def __init__(self, buy_order_id, buy_side, buy_price, buy_filled_quantity,buy_instrument,buy_timestamp):
#                  #sell_order_id, sell_side, sell_price, sell_filled_quantity,sell_instrument,sell_timestamp):
#         self.buy_order_id = buy_order_id
#         self.buy_side = buy_side
#         self.buy_price = buy_price
#         self.buy_filled_quantity = buy_filled_quantity
#         self.buy_instrument = buy_instrument
#         self.buy_timestamp = buy_timestamp
        
#         # self.sell_order_id = sell_order_id
#         # self.sell_side = sell_side
#         # self.sell_price = sell_price
#         # self.sell_filled_quantity = sell_filled_quantity
#         # self.sell_instrument = sell_instrument
#         # self.sell_timestamp = sell_timestamp

In [7]:
class MatchingEngine:
    def __init__(self):
        self.orderbooks = defaultdict(OrderBook)
        self.trades = defaultdict(list)
        self.buyerAckQueue = []
        self.sellerAckQueue = []
        self.orders = {}

    def placeOrder(self, instrument, side, price, quantity):
        order = Order(instrument, uuid.uuid1(), side, price, quantity, timestamp=int(time.time()))
        self.processOrder(order)
        self.orders[order.order_id] = order
        return order.order_id

    # Change the code to show order instrument,quantity,timestamp
    def getOrderBook(self, instrument):
        orderbook = self.orderbooks[instrument]
        # orderbook.showBestBidAndAsk()
        return orderbook.bids, orderbook.asks

    def showBestBidAsk(self, instrument):
        orderbook = self.orderbooks[instrument]
        orderbook.showBestBidAndAsk()

    def getTrades(self, instrument):
        return self.trades[instrument]

        
        
        
    def showTrades(self,instrument):
        trade_obj = self.getTrades(instrument)
        for i in trade_obj:
            trade_details = {
                "message" : f"Trades made for {i.name}",
                "order_id" : f"Order id {i.id}",
                "price" : f"{i.price}",
                "quantity traded": f"{i.volume}"    
            }
            
            # Convert the dictionary to a JSON-formatted string
            json_data_trade = json.dumps(trade_details, indent=2, default=self.uuid_serializer)

            # Print the JSON-formatted string
            print(json_data_trade)
        
        

    def processOrder(self, order):
        orderbook = self.orderbooks[order.instrument]

        timestamp = datetime.datetime.fromtimestamp(order.timestamp)

        if (order.side == 'buy'):
            print(f"{timestamp} : Buy order {order.order_id, order.instrument, order.price, order.quantity} is added to orderbook")


        else:
            print(f"{timestamp} : Sale order {order.order_id, order.instrument, order.price, order.quantity} is added to orderbook")


        if (order.side == 'buy' and orderbook.bestAsk() is not None and order.price >= orderbook.bestAsk().price):
            # Buy order crossed the spread
            self.matchBuyOrder(order, orderbook)
            # print(f"Buy order {order.order_id,order.instrument,order.price,order.quantity} matched with buy order {orderbook.bestAsk().order_id,orderbook.Ask().price,orderbook.bestAsk().quantity}")
            # print("----------------------------------------------------------------------------------------------------")


        elif (order.side == 'sell' and orderbook.bestBid() is not None and order.price <= orderbook.bestBid().price):
            # Sell order crossed the spread.
           
            self.matchSellOrder(order, orderbook)
            
        else:
            # Order did not cross the spread, place in order book
            orderbook.add(order)

   
    def uuid_serializer(self, obj):
        if isinstance(obj, uuid.UUID):
            return str(obj)
        raise TypeError(f'Object of type {obj.__class__.__name__} is not JSON serializable')

    def acknowledgeOrder(self, matchedOrder):
        timeACk = datetime.datetime.fromtimestamp(matchedOrder.timestamp)

        # Store the order details in a dictionary with the "message" key first
        order_details = {
            "message": f"Order acknowledgement for {'buy' if matchedOrder.side == 'buy' else 'sell'} order {matchedOrder.order_id, matchedOrder.instrument}",
            "order_id": matchedOrder.order_id,
            "instrument": matchedOrder.instrument,
            "price": matchedOrder.price,
            "filled_quantity": matchedOrder.filled_quantity,
            "timestamp": timeACk.isoformat(),
            "action": "Bought" if matchedOrder.side == "buy" else "Sold",
        }

        # Convert the dictionary to a JSON-formatted string
        json_data_ack = json.dumps(order_details, indent=2, default=self.uuid_serializer)

        # Print the JSON-formatted string
        print(json_data_ack)
        print("----------------------------------------------------------------------------------------------------")

        if matchedOrder.side == "buy":
            self.buyerAckQueue.append(matchedOrder)
        else:
            self.sellerAckQueue.append(matchedOrder)

    def cancelOrder(self, order_id):
        if order_id in self.orders:
            order = self.orders[order_id]
            if (order.quantity == 0):
                print(f"Cannot cancel order {order_id}, already fully filled")
                
            elif order.filled_quantity <= order.quantity:
                filled_qty = order.filled_quantity
                remaining_quantity = order.quantity
                order.quantity -= order.filled_quantity
                orderbook = self.orderbooks[order.instrument]
                orderbook.remove(order)
                del self.orders[order_id]

                print(
                    f"Order ID: {order_id} is cancelled. Filled quantity: {filled_qty}, Remaining quantity: {remaining_quantity}")


            else:
                orderbook = self.orderbooks[order.instrument]
                orderbook.remove(order)
                del self.orders[order_id]
                print(f"Order ID: {order_id} is cancelled")
        else:
            print("There is no such order")

    def match(self, order):
        orderbook = self.orderbooks[order.instrument]

        if (order.side == 'buy' and orderbook.bestAsk() is not None and order.price >= orderbook.bestAsk().price):
            # Buy order crossed the spread.
            self.matchBuyOrder(order, orderbook)
        elif (order.side == 'sell' and orderbook.bestBid() is not None and order.price <= orderbook.bestBid().price):
            # Sell order crossed the spread.
            self.matchSellOrder(order, orderbook)
        else:
            # # Order did not cross the spread, place in order book
            orderbook.add(order)

    def matchBuyOrder(self, order, orderbook):


        while orderbook.asks and order.price >= orderbook.bestAsk().price and order.quantity > 0:
                ask = orderbook.bestAsk()
                buyMatchedOrder = []

                if ask.quantity <= order.quantity:  # quantity for sale is less than quantity for buy

                    order.quantity -= ask.quantity
                    order.filled_quantity += ask.quantity
                    
                    # ask price is appended to trade since if ask price does not match buy, there will be no buy
                    trade = Trade(order.order_id, order.instrument, ask.price, order.filled_quantity)
                    self.trades[order.instrument].append(trade)

                    buyMatchedOrder.append(
                        MatchedOrder(ask.order_id, 'sell', ask.price, ask.quantity, ask.instrument, ask.timestamp))

                    
                    buyMatchedOrder.append( MatchedOrder(order.order_id, 'buy', ask.price, order.filled_quantity, order.instrument,order.timestamp))


                    self.acknowledgeOrder(buyMatchedOrder[0])
                    self.acknowledgeOrder(buyMatchedOrder[1])

                    # remove sell order as it is filled
                    orderbook.remove(ask)
                



                else:  # quantity for sale is more than quantity for buy

                    ask.quantity -= order.quantity
                    order.filled_quantity += order.quantity
                    ask.filled_quantity += order.filled_quantity
                    order.quantity = 0
                    
                    # ask price is appended to trade since if ask price does not match buy, there will be no buy
                    trade = Trade(order.order_id, order.instrument, ask.price, order.filled_quantity)
                    
                    self.trades[order.instrument].append(trade)
                    
                    buyMatchedOrder.appen(MatchedOrder(ask.order_id, 'sell', ask.price, ask.filled_quantity, ask.instrument, ask.timestamp))
                    buyMatchedOrder.append( MatchedOrder(order.order_id, 'buy', ask.price, order.filled_quantity, order.instrument,order.timestamp))


                    self.acknowledgeOrder(buyMatchedOrder[0])
                    self.acknowledgeOrder(buyMatchedOrder[1])

                    
                    orderbook.remove(order)

                # if buy order not fully filled add remaining back to orderbook
                if order.quantity > 0:
                    orderbook.add(order)

    def matchSellOrder(self, sellorder, orderbook):
        
       
        
        while orderbook.bids and sellorder.price <= orderbook.bestBid().price and sellorder.quantity > 0:
            bid = orderbook.bestBid()

            sellMatchedOrder = []
            
            # Buy Qty <= Sale Qty
            if bid.quantity <= sellorder.quantity:  
                sellorder.quantity -= bid.quantity
                sellorder.filled_quantity += bid.quantity
                bid.quantity = 0
                
                # trade executed at sell price
                trade = Trade(sellorder.order_id, sellorder.instrument, sellorder.price, sellorder.filled_quantity)
                self.trades[sellorder.instrument].append(trade)
                
                # buy filled quantity is sell filled quantity
                sellMatchedOrder.append(MatchedOrder(bid.order_id, 'buy', sellorder.price, sellorder.filled_quantity, bid.instrument, bid.timestamp))
                sellMatchedOrder.append(MatchedOrder(sellorder.order_id, 'sell', sellorder.price, sellorder.filled_quantity, sellorder.instrument,sellorder.timestamp))

                self.acknowledgeOrder(sellMatchedOrder[0])
                self.acknowledgeOrder(sellMatchedOrder[1])
                
                #buy order filled so remove from orderbook
                orderbook.removeafterTrade(bid)
                
             # Buy Qty > Sale Qty    
            else: 

                
               
                
                bid.quantity -= sellorder.quantity
               
                sellorder.filled_quantity += sellorder.quantity
                sellorder.quantity = 0
                
                
                # trade executed at sale price and sale filled quantity
                trade = Trade(sellorder.order_id, sellorder.instrument, sellorder.price, sellorder.filled_quantity)
                self.trades[sellorder.instrument].append(trade)
                
                sellMatchedOrder.append(MatchedOrder(bid.order_id, 'buy', sellorder.price, sellorder.filled_quantity, bid.instrument,bid.timestamp))
                sellMatchedOrder.append( MatchedOrder(sellorder.order_id, 'sell',sellorder.price, sellorder.filled_quantity, sellorder.instrument, sellorder.timestamp))

                

                self.acknowledgeOrder(sellMatchedOrder[0])
                self.acknowledgeOrder(sellMatchedOrder[1])
                
                # sale filled so remove from orderbook
                orderbook.removeafterTrade(sellorder)
                

        # If the sell order is not fully filled, add the remaining part to the order book
        if sellorder.quantity > 0:
            orderbook.add(sellorder)

### TESTING OUTPUT (DEBUGGING)

In [8]:
# my_order = Order('ETH', 12345,'buy', 150.0, 100, 1648763456)
# print("Instrument:", my_order.instrument)
# print("Order ID:", my_order.order_id)
# print("Side:", my_order.side)
# print("Price:", my_order.price)
# print("Quantity:", my_order.quantity)
# print("Timestamp:", my_order.timestamp)
# print("Filled Quantity:", my_order.filled_quantity)

# my_order2 = Order('ETH', 145,'buy', 230.0, 100, 1648763456)
# print("Instrument:", my_order.instrument)
# print("Order ID:", my_order.order_id)
# print("Side:", my_order.side)
# print("Price:", my_order.price)
# print("Quantity:", my_order.quantity)
# print("Timestamp:", my_order.timestamp)
# print("Filled Quantity:", my_order.filled_quantity)

# myOrderBook = OrderBook()
# myOrderBook.add(my_order)
# myOrderBook.add(my_order2)
# ord = myOrderBook.bestBid()
# print(ord.order_id)

### TEST CASES

#### Place buy order n retrieve orderbook

In [9]:
engine = MatchingEngine()

def show_trades(inst):
    trades = engine.getTrades(inst)
    
    if (len(trades)== 0 ):
        print(f"No trades were made for {inst}")
        print("No acknowledgments")
        return
        
    
        
    engine.showTrades(inst)

    print("Order acknowledgment for buyer and seller:")

    # Get filled orders.

    for i in trades:
        if engine.buyerAckQueue:
            buy_ack = engine.buyerAckQueue.pop()
            print("Buy filled:", buy_ack.order_id, buy_ack.side, buy_ack.instrument, buy_ack.price, buy_ack.filled_quantity)
        else:
            print("No buy acknowledgements")

        if engine.sellerAckQueue:
            sell_ack = engine.sellerAckQueue.pop()
            print("Sale filled:", sell_ack.order_id, sell_ack.side, sell_ack.price, sell_ack.filled_quantity)
        else:
            print("No sell acknowledgements")


In [10]:
instrument = "ETH"

In [11]:
buy_order_id1 = engine.placeOrder("ETH", "buy", 150, 2)

2023-05-02 23:06:33 : Buy order (UUID('ed280610-e8fa-11ed-ac80-9078418aa6e1'), 'ETH', 150, 2) is added to orderbook


In [12]:
sell_order_id1 = engine.placeOrder("ETH", "sell", 100, 4)

2023-05-02 23:06:35 : Sale order (UUID('ee3a23ab-e8fa-11ed-9e14-9078418aa6e1'), 'ETH', 100, 4) is added to orderbook
{
  "message": "Order acknowledgement for buy order (UUID('ed280610-e8fa-11ed-ac80-9078418aa6e1'), 'ETH')",
  "order_id": "ed280610-e8fa-11ed-ac80-9078418aa6e1",
  "instrument": "ETH",
  "price": 100,
  "filled_quantity": 2,
  "timestamp": "2023-05-02T23:06:33",
  "action": "Bought"
}
----------------------------------------------------------------------------------------------------
{
  "message": "Order acknowledgement for sell order (UUID('ee3a23ab-e8fa-11ed-9e14-9078418aa6e1'), 'ETH')",
  "order_id": "ee3a23ab-e8fa-11ed-9e14-9078418aa6e1",
  "instrument": "ETH",
  "price": 100,
  "filled_quantity": 2,
  "timestamp": "2023-05-02T23:06:35",
  "action": "Sold"
}
----------------------------------------------------------------------------------------------------


ValueError: list.remove(x): x not in list

In [ ]:

# buy_order_id1 = engine.placeOrder("ETH", "buy", 150, 2)



In [ ]:
engine.cancelOrder(sell_order_id1)

In [14]:
bids1, asks1 = engine.getOrderBook('ETH')
print("----------------------------------------------------------------------------------------------------")
print("OrderBook for ETH")
print("Bids:", bids1)
print("Asks:", asks1)
print("----------------------------------------------------------------------------------------------------")
engine.showBestBidAsk("ETH")

----------------------------------------------------------------------------------------------------
OrderBook for ETH
Bids: [(-150, 1683039993, UUID('ed280610-e8fa-11ed-ac80-9078418aa6e1'), 2, <__main__.Order object at 0x0000022319FEE310>)]
Asks: []
----------------------------------------------------------------------------------------------------
No best bid available.
----------------------------------------------------------------------------------------------------
No best ask available.
----------------------------------------------------------------------------------------------------


In [13]:
show_trades('ETH')
print("-------------------------------------------------------------------------------------------------------")

{
  "message": "Trades made for ETH",
  "order_id": "Order id ee3a23ab-e8fa-11ed-9e14-9078418aa6e1",
  "price": "100",
  "quantity traded": "2"
}
Order acknowledgment for buyer and seller:
Buy filled: ed280610-e8fa-11ed-ac80-9078418aa6e1 buy ETH 100 2
Sale filled: ee3a23ab-e8fa-11ed-9e14-9078418aa6e1 sell 100 2
-------------------------------------------------------------------------------------------------------
